In [1]:
import dropbox
from dropbox import DropboxOAuth2FlowNoRedirect

In [2]:
from pathlib import Path

In [3]:
import pprint

In [4]:
with open('/Users/jonnyhyman/data/dropbox_oath.txt','r') as f:
    readlist = f.readlines()
APP_KEY = readlist[0][:-1]
APP_SECRET = readlist[1]

In [5]:
'''
This example walks through a basic oauth flow using the existing long-lived token type
Populate your app key and app secret in order to run this locally
'''

auth_flow = DropboxOAuth2FlowNoRedirect(APP_KEY, APP_SECRET)

authorize_url = auth_flow.start()
print("1. Go to: " + authorize_url)
print("2. Click \"Allow\" (you might have to log in first).")
print("3. Copy the authorization code.")
auth_code = input("Enter the authorization code here: ").strip()

try:
    oauth_result = auth_flow.finish(auth_code)
except Exception as e:
    print('Error: %s' % (e,))
    exit(1)

with dropbox.Dropbox(oauth2_access_token=oauth_result.access_token) as dbx:
    dbx.users_get_current_account()
    print("Successfully set up client!")

1. Go to: https://www.dropbox.com/oauth2/authorize?response_type=code&client_id=gm6zp8ga0c6bn6w
2. Click "Allow" (you might have to log in first).
3. Copy the authorization code.


Enter the authorization code here:  iwYRC17HuekAAAAAAAGVuCEX1-Y2sauTvrkGuKa-X2s


Successfully set up client!


In [18]:
target = 'Veritasium/Godel/Drafts' # Don't prepend /

In [20]:
a = dbx.files_list_folder('/' + target)

for n, ai in enumerate(a.entries):
    print(str(n).ljust(3), '...', ai.name, ai.is_downloadable)

0   ... Godel - Large 540p.mov True
1   ... Godel v2.mov True
2   ... Godel v3.mov True
3   ... Godel v4 - Large 540p.mov True
4   ... end shot.mov True
5   ... Godel v5.mov True
6   ... Godel v6.mov True


In [21]:
dbx_root = Path("/Users/jonnyhyman/Dropbox")
a = dbx_root / target

for n, ai in enumerate(a.iterdir()):
    print(str(n).ljust(3), '...', ai.name)

0   ... Godel v6.mov
1   ... Godel v5.mov
2   ... Godel - Large 540p.mov
3   ... Godel v3.mov
4   ... end shot.mov
5   ... Godel v2.mov
6   ... Godel v4 - Large 540p.mov


In [22]:
lcl_path = Path("/Users/jonnyhyman/Dropbox")
rel_fold = target

In [28]:
dbx_files = [file for file in (dbx.files_list_folder('/'+target).entries)]
dbx_filen = [file.name for file in dbx_files]

In [35]:
lcl_files = [file for file in (dbx_root / Path(rel_fold)).iterdir()]
lcl_filen = [file.name for file in lcl_files]

In [38]:
lcl_filen

['Godel v6.mov',
 'Godel v5.mov',
 'Godel - Large 540p.mov',
 'Godel v3.mov',
 'end shot.mov',
 'Godel v2.mov',
 'Godel v4 - Large 540p.mov']

In [40]:
dbx_filen

['Godel - Large 540p.mov',
 'Godel v2.mov',
 'Godel v3.mov',
 'Godel v4 - Large 540p.mov',
 'end shot.mov',
 'Godel v5.mov',
 'Godel v6.mov']

In [30]:
if len(dbx_files) != len(lcl_filen):
    print("Not done syncing")

for fn, f in enumerate(lcl_filen):
    if (f not in dbx_filen):
        print(f"... {f} still syncing, not in Dropbox list yet")
    else:
        for d in dbx_files:
            if d.name == f and d.size==0:
                GB = lcl_files[fn].stat().st_size*1e-9
                print(f"... {f} still syncing, size is zero on server, {GB} GB on disk")

In [ ]:
dbx.files_list_folder(folder).entries[0]

In [ ]:
f = [i for i in dbx.files_list_folder(folder).entries if '005' in i.name][0]

In [ ]:
for f in dbx_files:
    print(f.name)

In [32]:
from time import sleep

In [42]:
while True:
    
    dbx_files = [file for file in (dbx.files_list_folder('/'+target).entries)]
    dbx_filen = [file.name for file in dbx_files]
    
    lcl_files = [file for file in (dbx_root / Path(rel_fold)).iterdir()]
    lcl_filen = [file.name for file in lcl_files]
    
    inc = 0
    
    # UPLOADS
    
    for fn, f in enumerate(lcl_filen):
        
        if (f not in dbx_filen):
            print(f"... {f} uploading, not in Dropbox list yet")
            inc += 1
        
        else:
            for d in dbx_files:
                print(d.name, d.size)
                if d.name == f and d.size==0:
                    GB = lcl_files[fn].stat().st_size*1e-9
                    print(f"... {f}  uploading, {GB} GB on disk")
                    inc += 1
    
    # DOWNLOADS
    
    for fn, f in enumerate(dbx_filen):
        
        if (f not in lcl_filen):
            print(f"... {f} downloading, not in folder yet")
            inc += 1
        
#         else:
#             for d in dbx_files:
                
#                 lcl_file = [lf for lf in lcl_files if lf.name == d.name][0]
#                 file_GB = lcl_file.stat().st_size*1e-9
                    
#                 if d.size*1e-9 < file_GB:
#                     print(f"... {f}  downloading, {round(file_GB,2)}GB of {round(d.size*1e-9,2)}GB")
#                     inc += 1
    
    if inc == 0:
        print("All files synced up and down!")
        break
        
    sleep(5*60)

Godel - Large 540p.mov 493365901
Godel v2.mov 1105559891
Godel v3.mov 1364041205
Godel v4 - Large 540p.mov 990145404
end shot.mov 233708393
Godel v5.mov 16771253450
Godel v6.mov 35103
Godel - Large 540p.mov 493365901
Godel v2.mov 1105559891
Godel v3.mov 1364041205
Godel v4 - Large 540p.mov 990145404
end shot.mov 233708393
Godel v5.mov 16771253450
Godel v6.mov 35103
Godel - Large 540p.mov 493365901
Godel v2.mov 1105559891
Godel v3.mov 1364041205
Godel v4 - Large 540p.mov 990145404
end shot.mov 233708393
Godel v5.mov 16771253450
Godel v6.mov 35103
Godel - Large 540p.mov 493365901
Godel v2.mov 1105559891
Godel v3.mov 1364041205
Godel v4 - Large 540p.mov 990145404
end shot.mov 233708393
Godel v5.mov 16771253450
Godel v6.mov 35103
Godel - Large 540p.mov 493365901
Godel v2.mov 1105559891
Godel v3.mov 1364041205
Godel v4 - Large 540p.mov 990145404
end shot.mov 233708393
Godel v5.mov 16771253450
Godel v6.mov 35103
Godel - Large 540p.mov 493365901
Godel v2.mov 1105559891
Godel v3.mov 1364041205